In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [5]:
# !pip install lightgbm==3.3.5

In [6]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [7]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [8]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [9]:
import os
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSEmploymentDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning

from source.preprocessing import get_simple_preprocessor, create_extra_test_sets
from source.experiment_interface import run_exp_iter_with_mult_set_and_preprocessing_intervention

## Define Input Variables

In [10]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'one_repair_lvl_many_models'
DB_COLLECTION_NAME = f'exp_{EXPERIMENT_NAME}'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.5]

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'folk_ny_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [11]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [12]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [13]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': '42eab82d-3396-491d-a457-6260b1caecae',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  42eab82d-3396-491d-a457-6260b1caecae


## Initialize custom objects

In [14]:
data_loader = ACSEmploymentDataset(state=['NY'], year=2018, with_nulls=False,
                                   subsample_size=20_000, subsample_seed=42)
data_loader.X_data.head()

,MAR,MIL,ESP,MIG,DREM,NATIVITY,DIS,DEAR,DEYE,SEX,RAC1P,RELP,CIT,ANC,SCHL,AGEP
0,5,4,0,1,2,1,2,2,2,1,1,2,1,1,15,42
1,1,4,0,1,2,1,2,2,2,2,1,0,1,2,21,59
2,2,4,0,1,2,2,2,2,2,2,1,0,4,4,19,78
3,5,4,0,1,2,1,2,2,2,1,1,10,1,4,14,19
4,1,2,0,1,2,1,2,2,2,1,1,0,1,1,17,87


In [15]:
data_loader.X_data.shape

(20000, 16)

In [16]:
data_loader2 = ACSEmploymentDataset(state=['NY'], year=2017, with_nulls=False,
                                    subsample_size=20_000, subsample_seed=42)
data_loader2.X_data.head()

,MAR,MIL,ESP,MIG,DREM,NATIVITY,DIS,DEAR,DEYE,SEX,RAC1P,RELP,CIT,ANC,SCHL,AGEP
0,1,4,0,1,2,2,2,2,2,1,1,1,5,1,19,45
1,1,4,0,3,2,2,2,2,2,2,6,0,4,1,22,38
2,5,4,0,1,2,1,2,2,2,1,1,2,1,1,21,26
3,5,4,0,1,2,2,2,2,2,2,6,0,5,1,16,36
4,1,4,0,1,2,1,2,2,2,2,1,2,1,1,16,42


In [17]:
data_loader2.X_data.shape

(20000, 16)

In [18]:
data_loader3 = ACSEmploymentDataset(state=['CA'], year=2018, with_nulls=False,
                                    subsample_size=20_000, subsample_seed=42)
data_loader3.X_data.head()

,MAR,MIL,ESP,MIG,DREM,NATIVITY,DIS,DEAR,DEYE,SEX,RAC1P,RELP,CIT,ANC,SCHL,AGEP
0,5,4,0,3,2,1,2,2,2,2,1,13,1,1,16,28
1,1,4,0,3,2,1,2,2,2,2,1,1,1,1,19,78
2,1,4,0,1,2,2,2,2,2,1,1,1,4,1,14,45
3,1,4,0,1,2,1,2,2,2,2,1,1,1,1,19,71
4,4,4,0,1,2,2,2,2,2,1,1,0,5,1,21,44


In [19]:
data_loader3.X_data.shape

(20000, 16)

In [20]:
data_loader4 = ACSEmploymentDataset(state=['CA'], year=2017, with_nulls=False,
                                    subsample_size=20_000, subsample_seed=42)
data_loader4.X_data.head()

,MAR,MIL,ESP,MIG,DREM,NATIVITY,DIS,DEAR,DEYE,SEX,RAC1P,RELP,CIT,ANC,SCHL,AGEP
0,1,4,0,1,2,2,2,2,2,2,6,0,4,1,21,34
1,3,4,0,1,2,1,2,2,2,1,1,6,1,1,21,61
2,1,4,0,1,2,2,2,2,2,1,9,0,4,1,1,66
3,1,4,0,1,2,1,2,2,2,1,1,1,1,1,19,64
4,1,4,0,1,2,2,1,2,2,2,1,1,5,1,9,62


In [21]:
data_loader4.X_data.shape

(20000, 16)

Create out-of-domain test sets

In [22]:
column_transformer = get_simple_preprocessor(data_loader)
extra_data_loaders = [data_loader2, data_loader3, data_loader4]
extra_test_sets = create_extra_test_sets(extra_data_loaders, column_transformer, TEST_SET_FRACTION, seed=42)

In [23]:
extra_test_sets[0][0].shape

(2000, 92)

## Run experiment iterations

### Experiment iteration 1

In [25]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
exp_iter_extra_test_sets = copy.deepcopy(extra_test_sets)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_mult_set_and_preprocessing_intervention(
    data_loader=exp_iter_data_loader,
    experiment_seed=experiment_seed,
    test_set_fraction=TEST_SET_FRACTION,
    db_writer_func=db_writer_func,
    fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
    extra_test_sets=exp_iter_extra_test_sets,
    column_transformer=preprocessor,
    models_params_for_tuning=models_params_for_tuning,
    metrics_computation_config=metrics_computation_config,
    custom_table_fields_dct=custom_table_fields_dct,
    with_tuning=True,
    # with_tuning=False,
    # tuned_params_df_paths=tuned_params_df_paths,
    save_results_dir_path=SAVE_RESULTS_DIR_PATH,
    verbose=True
)

2023-07-14 16:42:32 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[0.0, 0.5]',
 'model_init_seed': 100,
 'session_uuid': '42eab82d-3396-491d-a457-6260b1caecae'}




2023-07-14 16:42:33 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 2917,  2234, 14396,  1781, 11102,   732,  7692, 10589, 16098,
             4920,  6601,  7611,  3825, 18755,  6862,  3847,  7256, 13711,
            12389,  9772],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 2917,  2234, 14396,  1781, 11102,   732,  7692, 10589, 16098,
             4920,  6601,  7611,  3825, 18755,  6862,  3847,  7256, 13711,
            12389,  9772],
           dtype='int64')


Multiple alphas:   0%|          | 0/2 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [24]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230714__210625.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230714__221025.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
exp_iter_extra_test_sets = copy.deepcopy(extra_test_sets)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [25]:
run_exp_iter_with_mult_set_and_preprocessing_intervention(
    data_loader=exp_iter_data_loader,
    experiment_seed=experiment_seed,
    test_set_fraction=TEST_SET_FRACTION,
    db_writer_func=db_writer_func,
    fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
    extra_test_sets=exp_iter_extra_test_sets,
    column_transformer=preprocessor,
    models_params_for_tuning=models_params_for_tuning,
    metrics_computation_config=metrics_computation_config,
    custom_table_fields_dct=custom_table_fields_dct,
    # with_tuning=True,
    with_tuning=False,
    tuned_params_df_paths=tuned_params_df_paths,
    save_results_dir_path=SAVE_RESULTS_DIR_PATH,
    verbose=True
)

2023-07-14 19:31:09 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[0.0, 0.5]',
 'model_init_seed': 200,
 'session_uuid': '42eab82d-3396-491d-a457-6260b1caecae'}




2023-07-14 19:31:09 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 4298,  9201,   679, 18780,  6955,  8571,  9405, 13699,  3930,
            13573,  3774,  9085, 12646, 16782, 13102, 18789,  8042, 19647,
            13861,  8780],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 4298,  9201,   679, 18780,  6955,  8571,  9405, 13699,  3930,
            13573,  3774,  9085, 12646, 16782, 13102, 18789,  8042, 19647,
            13861,  8780],
           dtype='int64')


Multiple alphas:   0%|          | 0/2 [00:00<?, ?it/s]

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230714__210625.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230714__221025.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
exp_iter_extra_test_sets = copy.deepcopy(extra_test_sets)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_mult_set_and_preprocessing_intervention(
    data_loader=exp_iter_data_loader,
    experiment_seed=experiment_seed,
    test_set_fraction=TEST_SET_FRACTION,
    db_writer_func=db_writer_func,
    fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
    extra_test_sets=exp_iter_extra_test_sets,
    column_transformer=preprocessor,
    models_params_for_tuning=models_params_for_tuning,
    metrics_computation_config=metrics_computation_config,
    custom_table_fields_dct=custom_table_fields_dct,
    # with_tuning=True,
    with_tuning=False,
    tuned_params_df_paths=tuned_params_df_paths,
    save_results_dir_path=SAVE_RESULTS_DIR_PATH,
    verbose=True
)

### Experiment iteration 4

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230714__210625.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230714__221025.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
exp_iter_extra_test_sets = copy.deepcopy(extra_test_sets)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_mult_set_and_preprocessing_intervention(
    data_loader=exp_iter_data_loader,
    experiment_seed=experiment_seed,
    test_set_fraction=TEST_SET_FRACTION,
    db_writer_func=db_writer_func,
    fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
    extra_test_sets=exp_iter_extra_test_sets,
    column_transformer=preprocessor,
    models_params_for_tuning=models_params_for_tuning,
    metrics_computation_config=metrics_computation_config,
    custom_table_fields_dct=custom_table_fields_dct,
    # with_tuning=True,
    with_tuning=False,
    tuned_params_df_paths=tuned_params_df_paths,
    save_results_dir_path=SAVE_RESULTS_DIR_PATH,
    verbose=True
)

### Experiment iteration 5

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230714__210625.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230714__221025.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
exp_iter_extra_test_sets = copy.deepcopy(extra_test_sets)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_mult_set_and_preprocessing_intervention(
    data_loader=exp_iter_data_loader,
    experiment_seed=experiment_seed,
    test_set_fraction=TEST_SET_FRACTION,
    db_writer_func=db_writer_func,
    fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
    extra_test_sets=exp_iter_extra_test_sets,
    column_transformer=preprocessor,
    models_params_for_tuning=models_params_for_tuning,
    metrics_computation_config=metrics_computation_config,
    custom_table_fields_dct=custom_table_fields_dct,
    # with_tuning=True,
    with_tuning=False,
    tuned_params_df_paths=tuned_params_df_paths,
    save_results_dir_path=SAVE_RESULTS_DIR_PATH,
    verbose=True
)

### Experiment iteration 6

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230714__210625.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230714__221025.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
exp_iter_extra_test_sets = copy.deepcopy(extra_test_sets)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_mult_set_and_preprocessing_intervention(
    data_loader=exp_iter_data_loader,
    experiment_seed=experiment_seed,
    test_set_fraction=TEST_SET_FRACTION,
    db_writer_func=db_writer_func,
    fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
    extra_test_sets=exp_iter_extra_test_sets,
    column_transformer=preprocessor,
    models_params_for_tuning=models_params_for_tuning,
    metrics_computation_config=metrics_computation_config,
    custom_table_fields_dct=custom_table_fields_dct,
    # with_tuning=True,
    with_tuning=False,
    tuned_params_df_paths=tuned_params_df_paths,
    save_results_dir_path=SAVE_RESULTS_DIR_PATH,
    verbose=True
)

### Experiment iteration 7

### Experiment iteration 8

### Experiment iteration 9

### Experiment iteration 10